In [1]:
spark

org.apache.spark.sql.SparkSession@3fc7553d

# 이벤트 시간과 상태 기반 처리 
+ 이벤트 시간 처리 
    + 이벤트 시간 : 데이터에 기록된 시간 기준 처리 
    + 처리 시간 : 스트림 처리 시스템이 데이터를 실제로 수신한 시간 
+ 상태 기반 처리 
    + 중간 처리 정보를 사용하거나 갱신하는 경우에 필요(마이크로 배치 혹은 레코드 단위 처리) 
    + 스파크는 상태 저장소를 제공 
+ 임의적인 상태 기반 처리 
    + 상태의 유형, 갱신 방법, 제거 시점에 따라 세밀한 제어가 필요한 경우 사용 

## 22.4 이벤트 시간 처리의 기본  
+ Creation_Time: 이벤트가 생성된 시간 
+ Arrival_Time: 서버에 도착한 시간 

In [2]:
spark.conf.set("spark.sql.shuffle.partitions", 5) 

val activeDataPath = "../BookSamples/data/activity-data"
val static = spark.read.json(activeDataPath) 
val streaming = spark.readStream.schema(static.schema).option("maxFilesPerTrigger", 10).json(activeDataPath)

static.printSchema() 

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



activeDataPath = ../BookSamples/data/activity-data
static = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]
streaming = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]

In [3]:
spark.streams.active

Array()

## 22.5 이벤트 시간 윈도우 
+ 나노세컨드 단위의 유닉스 시간으로 표현됨

In [4]:
import org.apache.spark.sql.functions.{window, col}

val withEventTime = streaming.selectExpr("*", "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time") 

withEventTime = [Arrival_Time: bigint, Creation_Time: bigint ... 9 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 9 more fields]

### 22.5.1 텀블링 윈도우 
+ 디버깅 목적으로 결과를 메모리 싱크에 저장함 
+ 스트림 처리가 시작되면 SQL로 결과 조회가 가능함 
+ windows 컬럼은 struct 타입이며 윈도우 시작과 종료를 의미함 

In [29]:
val active = withEventTime.groupBy(window(col("event_time"), "10 minutes"))
    .count() 
    .writeStream
    .queryName("events_per_windows2") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7f1cefeb


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7f1cefeb

In [33]:
spark.sql("select * from events_per_windows2").show(20, 0) 

+------------------------------------------+------+
|window                                    |count |
+------------------------------------------+------+
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|150773|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|133323|
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|100853|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|99178 |
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|125679|
|[2015-02-24 13:10:00, 2015-02-24 13:20:00]|105494|
|[2015-02-23 10:30:00, 2015-02-23 10:40:00]|100443|
|[2015-02-23 10:40:00, 2015-02-23 10:50:00]|88681 |
|[2015-02-23 13:20:00, 2015-02-23 13:30:00]|106075|
|[2015-02-22 00:40:00, 2015-02-22 00:50:00]|35    |
|[2015-02-24 11:20:00, 2015-02-24 11:30:00]|113768|
|[2015-02-24 12:20:00, 2015-02-24 12:30:00]|133623|
|[2015-02-24 14:00:00, 2015-02-24 14:10:00]|150225|
|[2015-02-24 14:10:00, 2015-02-24 14:20:00]|169064|
|[2015-02-24 13:40:00, 2015-02-24 13:50:00]|132243|
|[2015-02-24 13:50:00, 2015-02-24 14:00:00]|96023 |
|[2015-02-23

In [34]:
active.stop()

In [35]:
spark.streams.active

Array()

+ 슬라이딩 윈도우 

In [10]:
import org.apache.spark.sql.functions.{window, col} 

val active = withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream 
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4ad7ef6


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4ad7ef6

In [12]:
spark.sql("select * from events_per_windows").show(20, 0) 

+------------------------------------------+------+
|window                                    |count |
+------------------------------------------+------+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|107668|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|150773|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|133323|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|35    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|100853|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|99178 |
|[2015-02-23 13:25:00, 2015-02-23 13:35:00]|91684 |
|[2015-02-24 14:25:00, 2015-02-24 14:35:00]|203945|
|[2015-02-23 12:55:00, 2015-02-23 13:05:00]|113953|
|[2015-02-22 00:40:00, 2015-02-22 00:50:00]|35    |
|[2015-02-23 12:35:00, 2015-02-23 12:45:00]|91221 |
|[2015-02-23 13:05:00, 2015-02-23 13:15:00]|205912|
|[2015-02-24 11:20:00, 2015-02-24 11:30:00]|113768|
|[2015-02-24 13:35:00, 2015-02-24 13:45:00]|174863|
|[2015-02-24 14:00:00, 2015-02-24 14:10:00]|150225|
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|125679|
|[2015-02-24

In [13]:
active.stop()

### 22.5.2 워터마크로 지연 데이터 제어하기 
+ 얼마나 늦게 도착한 데이터까지 받아들일지 기준을 정하지 않았기 때문에 중간 결과 데이터를 영원히 저장하는 문제가 있음 
+ 오래된 데이터를 제거하는 데 필요한 워터마크를 반드시 지정해야 함 
+ 최대 5시간까지 데이터가 지연된다면? 

In [36]:
val active = withEventTime 
    .withWatermark("event_time", "5 hours") 
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@29dac4ad


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@29dac4ad

In [38]:
active.stop()
spark.streams.active

Array()

## 22.6 스트림에서 중복 데이터 제거하기 
+ 상태 정보가 무한히 커지지 않도록 워터마크를 명시해야 함 
+ 예제에서는 중복 이벤트가 같은 식별자와 타임스탬프를 가진다고 가정함 

In [39]:
val active = withEventTime
    .withWatermark("event_time", "5 hours") 
    .dropDuplicates("User", "event_time") 
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes")) 
    .count() 
    .writeStream 
    .queryName("events_per_windows") 
    .format("memory") 
    .outputMode("complete") 
    .start() 

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@20af0964


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@20af0964

In [40]:
spark.sql("select * from events_per_windows").show(20, 0) 

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|13390|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|18932|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|16604|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|2    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|12754|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|12523|
|[2015-02-23 13:25:00, 2015-02-23 13:35:00]|11499|
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|15794|
|[2015-02-24 13:10:00, 2015-02-24 13:20:00]|13157|
|[2015-02-24 14:25:00, 2015-02-24 14:35:00]|25223|
|[2015-02-23 10:30:00, 2015-02-23 10:40:00]|12449|
|[2015-02-24 11:45:00, 2015-02-24 11:55:00]|17355|
|[2015-02-23 10:40:00, 2015-02-23 10:50:00]|11001|
|[2015-02-23 12:55:00, 2015-02-23 13:05:00]|14032|
|[2015-02-23 13:20:00, 2015-02-23 13:30:00]|13293|
|[2015-02-22 00:40:00, 2015-02-22 00:50:00]|2    |
|[2015-02-23 12:35:00, 2015-02-

In [41]:
active.stop()

In [42]:
spark.streams.active

Array()

## 22.7 임의적인 상태 기반 처리
+ 다음과 같은 처리 가능
    + 특정 키의 개수를 기반으로 윈도우 생성
    + 특성 시간 범위 안에 일정 개수 이상의 이벤트가 있는 경우 알림 발생시키기
    + 결정되지 않은 시간 동안 사용자 세션을 유지하고 향후 분석을 위해 세션 저장하기

+ 두 가지의 유형
    + mapGroupsWithState API : 최대 한 개의 로우를 만듬
    + flatMapGroupsWithState API : 하나 이상의 로우를 만듬

### 22.7.1 타임아웃
+ 타임 아웃은 전체 그룹에 대한 전역 파라미터로 동작함
+ 타임아웃은 처리 시간(GroupStateTimeout.ProcessingTimeTimeout)이나 이벤트 시간(GroupStateTimeout.EventTimeTimeout) 중 하나가 될 수 있음
+ state.hasTimedOut 값이나 values 이터레이터가 비어 있는지 확인하는 방식으로 타임아웃 정보 획득
+ 처리시간 기반인 경우 GroupState.setTimeoutDuration 설정 가능

### 22.7.2 출력 모드
+ mapGroupsWithState는 update 모드만 지원
+ flatMapGroupsWithState 은 update와 append 모드를 사용할 수 있음
+ append 모드는 타임아웃 이후에 결과 셋에서 데이터를 볼 수 있음

### 22.7.3 mapGroupsWithState
+ 필요 요소
    + 세 가지 클래스 정의 : 입력 클래스, 상태 클래스, 출력 클래스(선택)
    + 키, 이벤트 이터레이터 그리고 이전 상태를 기반으로 상태를 갱신하는 함수
    + 타임아웃 파라미터 (22.7.1절)

In [43]:
static.count()

6240991

In [45]:
static.show(5, 0)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x           |y           |z           |
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|18   |nexus4|g   |stand|3.356934E-4 |-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|66   |nexus4|g   |stand|-0.005722046|0.029083252 |0.005569458 |
|1424686735500|1424686733498505625|nexus4_1|99   |nexus4|g   |stand|0.0078125   |-0.017654419|0.010025024 |
|1424686735691|1424688581745026978|nexus4_2|145  |nexus4|g   |stand|-3.814697E-4|0.0184021   |-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|185  |nexus4|g   |stand|-3.814697E-4|-0.031799316|-0.00831604 |
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 5 rows



In [66]:
import org.apache.spark.sql.functions.{first, col}

static.groupBy("User").count().sort("User").show()

+----+------+
|User| count|
+----+------+
|   a|646829|
|   b|729907|
|   c|617237|
|   d|649961|
|   e|768182|
|   f|736442|
|   g|733387|
|   h|618617|
|   i|740429|
+----+------+



In [47]:
case class InputRow(user:String, timestamp:java.sql.Timestamp, activity:String)
case class UserState(user:String,
                     var activity:String,
                     var start:java.sql.Timestamp,
                     var end:java.sql.Timestamp)

defined class InputRow
defined class UserState


In [48]:
// 특정 행동을 얼마나 지속하고 있는 지 사용자 상태를 업데이트
def updateUserStateWithEvent(state:UserState, input:InputRow):UserState = {
    if (Option(input.timestamp).isEmpty){
        return state
    }
    if (state.activity == input.activity){
        state.end = input.timestamp
        
        if (input.timestamp.after(state.end)){
            state.end = input.timestamp
        }
        if (input.timestamp.before(state.start)){
            state.start = input.timestamp
        }
    }
    else {
        if (input.timestamp.after(state.end)){
            state.start = input.timestamp
            state.end = input.timestamp
            state.activity = input.activity
        }
    }
    
    state
}

updateUserStateWithEvent: (state: UserState, input: InputRow)UserState


In [49]:
import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}

def updateAcrossEvents(user:String,
                       inputs:Iterator[InputRow],
                       oldState: GroupState[UserState]):UserState = {
    
    var state:UserState = if (oldState.exists) oldState.get
                          else UserState(user, "",
                                         new java.sql.Timestamp(6284160000000L), // 초기에 무조건 업데이트 되도록 
                                         new java.sql.Timestamp(6284160L))       // 시간시간과 종료시간을 Max로 뒤집음
    // 시간 비교를 위해 이전 날짜를 간단하게 지정하고 데이터의 값을 기준으로 즉시 변경
    for (input <- inputs) {
        state = updateUserStateWithEvent(state, input)
        oldState.update(state)
    }
    state
}

updateAcrossEvents: (user: String, inputs: Iterator[InputRow], oldState: org.apache.spark.sql.streaming.GroupState[UserState])UserState


In [60]:
import org.apache.spark.sql.streaming.GroupStateTimeout

val active = withEventTime
    .selectExpr("User as user", "cast(Creation_time/1000000000 as timestamp) as timestamp", "gt as activity")
    .as[InputRow]
    .groupByKey(_.user)
    .mapGroupsWithState(GroupStateTimeout.NoTimeout)(updateAcrossEvents)
    .writeStream
    .queryName("events_per_window")
    .format("memory")
    .outputMode("update")
    .start()

active = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@23b33a23


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@23b33a23

In [64]:
spark.sql("SELECT * FROM events_per_window order by user, end").show(20, 0) 

+----+--------+--------------------------+--------------------------+
|user|activity|start                     |end                       |
+----+--------+--------------------------+--------------------------+
|a   |bike    |2015-02-23 13:30:15.772492|2015-02-23 13:35:15.285731|
|a   |bike    |2015-02-23 13:30:15.787598|2015-02-23 13:35:15.295954|
|a   |bike    |2015-02-23 13:30:15.837396|2015-02-23 13:35:15.351252|
|a   |bike    |2015-02-23 13:30:15.792481|2015-02-23 14:06:02.857617|
|a   |bike    |2015-02-23 13:30:16.230041|2015-02-23 14:06:02.882794|
|b   |bike    |2015-02-24 14:01:44.679329|2015-02-24 14:08:09.5051  |
|b   |bike    |2015-02-24 14:01:45.298684|2015-02-24 14:08:09.535312|
|b   |bike    |2015-02-24 14:01:44.105233|2015-02-24 14:08:09.565524|
|b   |bike    |2015-02-24 14:01:44.331887|2015-02-24 14:08:09.616062|
|b   |bike    |2015-02-24 14:01:44.346871|2015-02-24 14:08:09.641238|
|c   |bike    |2015-02-23 12:40:27.495645|2015-02-23 13:15:54.817554|
|c   |bike    |2015-

In [65]:
spark.sql("SELECT * FROM events_per_window order by user, start").count()

72

In [70]:
active.stop
active.isActive

false

#### 예제 : 카운트 기반 윈도우

+ 500개씩 누적될 때마다 결과를 출력
    + device와 timestamp 필드가 있는 입력용 케이스 클래스
    + 수집된 레코드의 현재 수, 장비 ID, 그리고 윈도우에서 읽은 이벤트의 배열을 가진 상태 케이스 클래스
    + 출력용 케이스 클래스

In [6]:
case class InputRow(device: String, timestamp: java.sql.Timestamp, x: Double)
case class DeviceState(device: String, var values: Array[Double], var count: Int)
case class OutputRow(device: String, previousAverage: Double)

defined class InputRow
defined class DeviceState
defined class OutputRow


In [7]:
def updateWithEvent(state:DeviceState, input:InputRow):DeviceState = {
    state.count += 1
    
    state.values = state.values ++ Array(input.x)
    state
}

updateWithEvent: (state: DeviceState, input: InputRow)DeviceState


In [8]:
import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}

def updateAcrossEvents(device:String,
                       inputs:Iterator[InputRow],
                       oldState:GroupState[DeviceState]):Iterator[OutputRow] = {
    
    inputs.toSeq.sortBy(_.timestamp.getTime).toIterator.flatMap { input =>
        val state = if (oldState.exists) oldState.get
                    else DeviceState(device, Array(), 0)
        
        val newState = updateWithEvent(state, input)
        if (newState.count >= 500) {
            // 윈도우 중 하나가 완료되면 빈 DeviceState로 교체
            // 과거 상태값에서 지난 500개 아이템의 평균을 구한 후 출력
            oldState.update(DeviceState(device, Array(), 0))
            Iterator(OutputRow(device,
                               newState.values.sum / newState.values.length.toDouble))
        }
        else {
            // 현재 DeviceState 객체로 업데이트하며 레코드를 출력하지 않음
            oldState.update(newState)
            Iterator()
        }
    }
}

updateAcrossEvents: (device: String, inputs: Iterator[InputRow], oldState: org.apache.spark.sql.streaming.GroupState[DeviceState])Iterator[OutputRow]


In [9]:
import org.apache.spark.sql.streaming.GroupStateTimeout

val activeStreamQuery = withEventTime
    .selectExpr("Device as device", "cast(Creation_time/1000000000 as timestamp) as timestamp", "x")
    .as[InputRow]
    .groupByKey(_.device)
    .flatMapGroupsWithState(OutputMode.Append,GroupStateTimeout.NoTimeout)(updateAcrossEvents)
    .writeStream
    .queryName("count_based_device")
    .format("memory")
    .outputMode("append")
    .start()

activeStreamQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1f5eead0


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1f5eead0

In [12]:
spark.sql("SELECT * FROM count_based_device").show(20, 0)

+--------+----------------------+
|device  |previousAverage       |
+--------+----------------------+
|nexus4_1|9.618530235999994E-4  |
|nexus4_1|-6.832885523999994E-4 |
|nexus4_1|3.549194291999999E-4  |
|nexus4_1|-6.747436750000009E-4 |
|nexus4_1|4.38232380200002E-4   |
|nexus4_1|9.082031224E-4        |
|nexus4_1|4.937744130000001E-4  |
|nexus4_1|1.6693114360000024E-4 |
|nexus4_1|-7.836914039999995E-4 |
|nexus4_1|6.668090874000004E-4  |
|nexus4_1|-1.2573242699999968E-4|
|nexus4_1|6.86035155E-4         |
|nexus4_1|3.0578613100000035E-4 |
|nexus4_1|0.007859497038599986  |
|nexus4_1|0.001040649405199999  |
|nexus4_1|-6.74743674200001E-4  |
|nexus4_1|-6.597900420000009E-4 |
|nexus4_1|-1.2786865039999968E-4|
|nexus4_1|6.439209080000028E-5  |
|nexus4_1|7.037352340000015E-5  |
+--------+----------------------+
only showing top 20 rows



In [13]:
spark.streams.active

Array(org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1f5eead0)

In [14]:
activeStreamQuery.runId

d9e3b9ec-c0e3-4b9e-993d-c1c43cc1a37d

In [15]:
activeStreamQuery.isActive

true

In [16]:
activeStreamQuery.stop
activeStreamQuery.isActive

false

### 22.7.4 flatMapGroupsWithState
+ 필요 요소
    + 세 가지 클래스 정의 : 입력 클래스, 상태 클래스, 출력 클래스(선택)
    + 키, 이벤트 이터레이터 그리고 이전 상태를 기반으로 상태를 갱신하는 함수
    + 타임아웃 파라미터 (22.7.1절)

#### 예제 : 세션화
+ 사용자 ID와 일부 시간정보를 이용해 실행 시점에서 세션을 생성함
+ 5초안에 해당 사용자가 새로운 이벤트를 만들지 않으면 세션을 종료

In [18]:
case class InputRow(uid:String, timestamp:java.sql.Timestamp, x:Double, activity:String)
case class UserSession(val uid:String, var timestamp:java.sql.Timestamp, var activities: Array[String], var values: Array[Double])
case class UserSessionOutput(val uid:String, var activities: Array[String], var xAvg:Double)

defined class InputRow
defined class UserSession
defined class UserSessionOutput


In [19]:
def updateWithEvent(state:UserSession, input:InputRow):UserSession = {
    // 비정상 날짜
    if (Option(input.timestamp).isEmpty) {
        return state
    }
    
    state.timestamp = input.timestamp
    state.values = state.values ++ Array(input.x)
    if (!state.activities.contains(input.activity)) {
      state.activities = state.activities ++ Array(input.activity)
    }
    state
}

updateWithEvent: (state: UserSession, input: InputRow)UserSession


In [20]:
import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}

def updateAcrossEvents(uid:String, inputs: Iterator[InputRow],
    oldState: GroupState[UserSession]):Iterator[UserSessionOutput] = {

    inputs.toSeq.sortBy(_.timestamp.getTime).toIterator.flatMap { input =>
        val state = if (oldState.exists) oldState.get
                    else UserSession(uid,
                                     new java.sql.Timestamp(6284160000000L),
                                     Array(),
                                     Array())
        val newState = updateWithEvent(state, input)
        if (oldState.hasTimedOut) {
            val state = oldState.get
            oldState.remove()
            Iterator(UserSessionOutput(uid,
                                       state.activities,
                                       newState.values.sum / newState.values.length.toDouble))
        }
        else if (state.values.length > 1000) {
            val state = oldState.get
            oldState.remove()
            Iterator(UserSessionOutput(uid,
                                       state.activities,
                                       newState.values.sum / newState.values.length.toDouble))
        }
        else {
            oldState.update(newState)
            oldState.setTimeoutTimestamp(newState.timestamp.getTime(), "5 seconds") // 5초 늦게 도착한 이벤트까지만 처리
            Iterator()
        }
    }
}

updateAcrossEvents: (uid: String, inputs: Iterator[InputRow], oldState: org.apache.spark.sql.streaming.GroupState[UserSession])Iterator[UserSessionOutput]


In [21]:
import org.apache.spark.sql.streaming.GroupStateTimeout

val activeQuery = withEventTime.where("x is not null")
    .selectExpr("user as uid",
                "cast(Creation_Time/1000000000 as timestamp) as timestamp",
                "x", "gt as activity")
    .as[InputRow]
    .withWatermark("timestamp", "5 seconds")
    .groupByKey(_.uid)
    .flatMapGroupsWithState(OutputMode.Append,
                            GroupStateTimeout.EventTimeTimeout)(updateAcrossEvents)
    .writeStream
    .queryName("count_based_device")
    .format("memory")
    .start()

activeQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3a3c6256


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3a3c6256

In [24]:
spark.sql("SELECT * FROM count_based_device").show(20, 0)

+---+------------------+----------------------+
|uid|activities        |xAvg                  |
+---+------------------+----------------------+
|a  |[stand]           |0.001636105028871131  |
|a  |[stand]           |-0.0023759961309690303|
|a  |[stand]           |3.791222360639342E-4  |
|a  |[stand]           |-0.003535877572927077 |
|a  |[stand]           |0.0017129324781218753 |
|a  |[stand]           |-3.7026542657342884E-5|
|a  |[stand, null, sit]|-0.0028615030874125846|
|a  |[sit]             |1.7531602997002986E-4 |
|a  |[sit]             |-4.881745294705291E-4 |
|a  |[sit]             |-2.7621303606393583E-4|
|a  |[sit]             |1.4277103326673296E-4 |
|a  |[sit]             |8.940339170829161E-5  |
|a  |[sit]             |1.7837997412587433E-4 |
|a  |[sit]             |4.268186813187034E-6  |
|a  |[sit, null]       |-1.0136957722277705E-4|
|a  |[null]            |-0.03021667061188812  |
|a  |[null, walk]      |-0.06103302192717283  |
|a  |[walk]            |-0.0263902360050

In [25]:
activeQuery.isActive

true

In [27]:
activeQuery.status

{
  "message" : "Waiting for data to arrive",
  "isDataAvailable" : false,
  "isTriggerActive" : false
}


In [28]:
activeQuery.recentProgress

Array({
  "id" : "cd5a2bdf-ac33-43dd-966f-83f376e4cd69",
  "runId" : "7b0462bd-0438-4f86-afdc-bf481046e1b5",
  "name" : "count_based_device",
  "timestamp" : "2020-06-27T02:47:13.864Z",
  "batchId" : 0,
  "numInputRows" : 780119,
  "processedRowsPerSecond" : 157313.77293809236,
  "durationMs" : {
    "addBatch" : 4723,
    "getBatch" : 10,
    "getOffset" : 48,
    "queryPlanning" : 128,
    "triggerExecution" : 4959,
    "walCommit" : 30
  },
  "eventTime" : {
    "avg" : "2015-02-24T03:00:44.892Z",
    "max" : "2015-02-24T15:21:45.941Z",
    "min" : "2015-02-22T00:41:40.120Z",
    "watermark" : "1970-01-01T00:00:00.000Z"
  },
  "stateOperators" : [ {
    "numRowsTotal" : 9,
    "numRowsUpdated" : 9,
    "memoryUsedB...


In [29]:
activeQuery.stop
activeQuery.isActive

false